Given a list of users and their friendships, find the users who have the most friends in common

# Part 1 - Finding Key Connectors

In [2]:
users = [
    { "id": 0, "name": "Hero" },
    { "id": 1, "name": "Dunn" },
    { "id": 2, "name": "Sue" },
    { "id": 3, "name": "Chi" },
    { "id": 4, "name": "Thor" },
    { "id": 5, "name": "Clive" },
    { "id": 6, "name": "Hicks" },
    { "id": 7, "name": "Devin" },
    { "id": 8, "name": "Kate" },
    { "id": 9, "name": "Klein" }
]

friendship_pairs = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4), (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

Having friendships represented as a list of pairs is not the easiest way to work with
them. To find all the friendships for user 1, you have to iterate over every pair looking
for pairs containing 1. If you had a lot of pairs, this would take a long time.
Instead, let’s create a dict where the keys are user ids and the values are lists of friend
ids. (Looking things up in a dict is very fast.)

In [3]:
# Initialize the dict with an empty list for each user id:
friendships = {user["id"]: [] for user in users}

# And then populate it with the friendships data:
for i, j in friendship_pairs:
    friendships[i].append(j) # Add j as a friend of user i
    friendships[j].append(i) # Add i as a friend of user j

print(friendships)

{0: [1, 2], 1: [0, 2, 3], 2: [0, 1, 3], 3: [1, 2, 4], 4: [3, 5], 5: [4, 6, 7], 6: [5, 8], 7: [5, 8], 8: [6, 7, 9], 9: [8]}


Now we can ask questions of our graph, like “what’s the average number of connections?”:

In [7]:
def number_of_friends(user):
    user_id = user["id"]
    friend_ids = friendships[user_id]
    return len(friend_ids)

total_connections = sum(number_of_friends(user) for user in users) # 24

num_users = len(users) # length of the users list # 10
avg_connections = total_connections / num_users 

print(avg_connections)

2.4


Find the most connected people

In [9]:
# Create a list (user_id, number_of_friends)
num_friends_by_id = [(user["id"], number_of_friends(user)) for user in users]

num_friends_by_id.sort( # Sort the list
    key=lambda id_and_friends: id_and_friends[1], # by num_friends
    reverse=True # largest to smallest
)

print(num_friends_by_id) # Each pair is (user_id, num_friends)

[(1, 3), (2, 3), (3, 3), (5, 3), (8, 3), (0, 2), (4, 2), (6, 2), (7, 2), (9, 1)]


# Part 2 - "Data Scientists You May Know" suggester

Create the function to iterate over friends and collect friends of friends.

In [17]:
def foaf_ids_bad(user):
    """foaf is short for 'friend of a friend' """
    return [foaf_id
            for friend_id in friendships[user["id"]]
            for foaf_id in friendships[friend_id]]

print(foaf_ids_bad(users[0]))

[0, 2, 3, 0, 1, 3]


Knowing that people are friends of friends in multiple ways seems like interesting
information, so maybe instead we should produce a count of mutual friends. And we
should probably exclude people already known to the user:

In [18]:
from collections import Counter

def friends_of_friends(user):
    user_id = user["id"]
    return Counter(
        foaf_id
        for friend_id in friendships[user_id] # For each of my friends
        for foaf_id in friendships[friend_id] # Find their friends
        if foaf_id != user_id # Who aren't me
        and foaf_id not in friendships[user_id] # And aren't my friends
    )

print(friends_of_friends(users[3]))

# This correctly tells Chi (id 3) that she has two mutual friends with Hero (id 0) but only one mutual friend with Clive (id 5).

Counter({0: 2, 5: 1})


As a data scientist, you know that you also might enjoy meeting users with similar
interests. (This is a good example of the “substantive expertise” aspect of data science.)
After asking around, you manage to get your hands on this data, as a list of
pairs (user_id, interest):

In [19]:
interests = [
(0, "Hadoop"), (0, "Big Data"), (0, "HBase"), (0, "Java"),
(0, "Spark"), (0, "Storm"), (0, "Cassandra"),
(1, "NoSQL"), (1, "MongoDB"), (1, "Cassandra"), (1, "HBase"),
(1, "Postgres"), (2, "Python"), (2, "scikit-learn"), (2, "scipy"),
(2, "numpy"), (2, "statsmodels"), (2, "pandas"), (3, "R"), (3, "Python"),
(3, "statistics"), (3, "regression"), (3, "probability"),
(4, "machine learning"), (4, "regression"), (4, "decision trees"),
(4, "libsvm"), (5, "Python"), (5, "R"), (5, "Java"), (5, "C++"),
(5, "Haskell"), (5, "programming languages"), (6, "statistics"),
(6, "probability"), (6, "mathematics"), (6, "theory"),
(7, "machine learning"), (7, "scikit-learn"), (7, "Mahout"),
(7, "neural networks"), (8, "neural networks"), (8, "deep learning"),
(8, "Big Data"), (8, "artificial intelligence"), (9, "Hadoop"),
(9, "Java"), (9, "MapReduce"), (9, "Big Data")
]

# For example, Hero (id 0) has no friends in common with Klein (id 9), but they share interests in Java and big data.
# It is easy to build a function that finds users with a certain interest:

def data_scientists_who_like(target_interest):
    return [user_id
            for user_id, user_interest in interests
            if user_interest == target_interest]

print(data_scientists_who_like("Java"))

[0, 5, 9]


This works, but it has to examine the whole list of interests for every search. If we
have a lot of users and interests (or if we just want to do a lot of searches), we’re probably
better off building an index from interests to users:

In [25]:
from collections import defaultdict

# Keys are interests, values are lists of user_ids with that interest
user_ids_by_interest = defaultdict(list)

for user_id, interest in interests:
    user_ids_by_interest[interest].append(user_id) # We get: {'Java': [0, 5, 9], 'Big Data': [0, 8, 9], ...}

# And another from users to interests
# Keys are user_ids, values are lists of interests for that user_id
interests_by_user_id = defaultdict(list)

for user_id, interest in interests:
    interests_by_user_id[user_id].append(interest) # We get: {0: ['Hadoop', 'Big Data', 'HBase', 'Java', 'Spark', 'Storm', 'Cassandra'], ...}

Now it's easy to find who has the most interests in common with a given user.
- iterate over the user's interests
- for each interest, iterate over the users with that interest
- keep count of how many times we see each user

In [26]:
def most_common_interests_with(user):
    return Counter(
        interested_user_id
        for interest in interests_by_user_id[user["id"]]
        for interested_user_id in user_ids_by_interest[interest]
        if interested_user_id != user["id"]
    )

print(most_common_interests_with(users[0])) # User 9 has 3 common interests with user 0

Counter({9: 3, 1: 2, 8: 1, 5: 1})
